In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [6]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [7]:
result = model(tokens)

In [8]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [9]:
int(torch.argmax(result.logits))+1

4

In [23]:
r = requests.get('https://www.yelp.com/biz/uka-los-angeles?osq=Japanese')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [24]:
reviews

['Uka is the shortened name of\xa0Uka no Mitama-no-Kami, the god of grain and fertility in the ancient native Shintō religion.\xa0Uka resides in the elegant, Japanese minimalist space\xa0designed by Ryū Kōsaka\xa0with large windows offering a 5 stories high panoramic view of Hollywood\xa0 and Downtown L.A. skyscrapers.\xa0Uka inherited this space from the former Inn Ann Restaurant which hosted many pop ups featuring noted Japanese chefs such as Hiro\'o Nagahara, Katsumi Kusumoto,\xa0David Schlosser, etc. and hosted the 2022 Rāmen Series where one could sample various Rāmen weekly from all over Japan by visiting chefs all under the aegis of Japan House, an organization that promotes Japanese Culture and Arts.\xa0 \xa0Kaisèki is an elegant multi-course menu dégustation that originated as a simple meal from Sadō, Japanese Zen Buddhist\xa0 Tea Ceremony which is a form of meditation. After many centuries, the simple meal has evolved into an elaborated state that still hews to the original p

In [25]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [26]:
df['review'].iloc[0]

'Uka is the shortened name of\xa0Uka no Mitama-no-Kami, the god of grain and fertility in the ancient native Shintō religion.\xa0Uka resides in the elegant, Japanese minimalist space\xa0designed by Ryū Kōsaka\xa0with large windows offering a 5 stories high panoramic view of Hollywood\xa0 and Downtown L.A. skyscrapers.\xa0Uka inherited this space from the former Inn Ann Restaurant which hosted many pop ups featuring noted Japanese chefs such as Hiro\'o Nagahara, Katsumi Kusumoto,\xa0David Schlosser, etc. and hosted the 2022 Rāmen Series where one could sample various Rāmen weekly from all over Japan by visiting chefs all under the aegis of Japan House, an organization that promotes Japanese Culture and Arts.\xa0 \xa0Kaisèki is an elegant multi-course menu dégustation that originated as a simple meal from Sadō, Japanese Zen Buddhist\xa0 Tea Ceremony which is a form of meditation. After many centuries, the simple meal has evolved into an elaborated state that still hews to the original pr

In [27]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [28]:
sentiment_score(df['review'].iloc[1])

5

In [29]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [30]:
df

,review,sentiment
0,Uka is the shortened name of Uka no Mitama-no-...,4
1,I'm not sure how to thank you Mr. Gary. Your ...,5
2,I have been to Japan House for events before. ...,4
3,"Thank you so much, Christina! We loved having ...",5
4,Parking:We took an Uber because we were drinki...,4
5,"Thank you kindly for your detailed review, Lis...",5
6,The view here is a total knockout and the serv...,3
7,Thank you so much for your detailed review. W...,5
8,This was one of the most exquisitely delicious...,5
9,Thank you sincerely for sharing your experienc...,5


In [22]:
df['review'].iloc[3]

'This baklava is the best I have ever had.  Their family recipe is fantastic!!  On top of that their customer service is phenomenal and they are happy to answer questions if you have them and even hand delivered product to us.'